In [1]:
import os
import pandas as pd
from google.cloud import storage
import requests

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.environ.get('GOOGLE_CREDENTIALS')
BUCKET = os.environ.get("GCP_GCS_BUCKET", 'dezoomcamp_2024_storage_bucket_radiant-gateway-412001')

init_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
#init_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet'

In [2]:
def download_dataset(url, target):
    r = requests.get(url)
    open(target, 'wb').write(r.content)
            
def web_to_gcs_parquet(year, service):
    for i in range(1, 13):
        month = f'0{i}'
        month = month[-2:]
        file_name = f'{service}_tripdata_{year}-{month}.parquet'
        
        request_url = f'{init_url}{file_name}'
        download_dataset(request_url, file_name)
        upload_csv_to_gcs(BUCKET, f'{service}_{year}_parquet/{file_name}', file_name)

def upload_csv_to_gcs(bucket_name, object_name, local_file):
    storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
    
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)


In [3]:
web_to_gcs_parquet(2022, 'green')